In [98]:
import pandas as pd
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_relevant_features

In [2]:
cholera_case_crosstab = pd.read_csv(r'C:\Users\Rohil\Documents\iGEM\yemen\cholera_epi_data\yemen_cholera_case_data_differenced.csv', dayfirst = True)
conflict_crosstab = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/fatality_crosstab.csv')
cholera_death_crosstab = pd.read_csv(r'C:\Users\Rohil\Documents\iGEM\yemen\cholera_epi_data\yemen_cholera_death_data_differenced.csv', dayfirst = True)
max_rainfall_crosstab = pd.read_csv(r'C:\Users\Rohil\Documents\iGEM\yemen\rainfall\yemen_daily_max_rainfall_crosstab.csv', dayfirst = True)
mean_rainfall_crosstab = pd.read_csv(r'C:\Users\Rohil\Documents\iGEM\yemen\rainfall\yemen_daily_mean_rainfall_crosstab.csv', dayfirst = True)

In [3]:
gov_pop_area_data = pd.read_excel('/Users/Rohil/Documents/iGEM/yemen/gov_area_pop_data.xlsx')
gov_pop_area_data = gov_pop_area_data[gov_pop_area_data.iso != 'YE-HD']

In [9]:
norm_cholera_case_crosstab = cholera_case_crosstab
for index, row in gov_pop_area_data[['iso', 'population']].iterrows():
    norm_cholera_case_crosstab[row.iso] = (norm_cholera_case_crosstab[row.iso] * 10000) / row.population

In [10]:
norm_cholera_death_crosstab = cholera_death_crosstab
for index, row in gov_pop_area_data[['iso', 'population']].iterrows():
    norm_cholera_death_crosstab[row.iso] = (norm_cholera_death_crosstab[row.iso] * 10000) / row.population

In [13]:
norm_conflict_crosstab = conflict_crosstab
for index, row in gov_pop_area_data[['iso', 'population']].iterrows():
    norm_conflict_crosstab[row.iso] = (norm_conflict_crosstab[row.iso] * 10000) / row.population

In [11]:
cases_unstacked = norm_cholera_case_crosstab.set_index('date').unstack().reset_index()
cases_unstacked.columns = ['gov_iso', 'date', 'new_cases']

In [12]:
deaths_unstacked = norm_cholera_death_crosstab.set_index('date').unstack().reset_index()
deaths_unstacked.columns = ['gov_iso', 'date', 'new_deaths']

In [21]:
fatalities_unstacked = norm_conflict_crosstab.set_index('date').unstack().reset_index()
fatalities_unstacked.columns = ['gov_iso', 'date', 'fatalities']

In [15]:
mean_rainfall_unstacked = mean_rainfall_crosstab.set_index('date').unstack().reset_index()
mean_rainfall_unstacked.columns = ['gov_iso', 'date', 'mean_rainfall']

In [16]:
max_rainfall_unstacked = max_rainfall_crosstab.set_index('date').unstack().reset_index()
max_rainfall_unstacked.columns = ['gov_iso', 'date', 'max_rainfall']

In [24]:
cases_unstacked.date = pd.to_datetime(cases_unstacked.date, format = "%d-%m-%y")

In [26]:
deaths_unstacked.date = pd.to_datetime(deaths_unstacked.date, format = "%d-%m-%y")

In [28]:
mean_rainfall_unstacked.date = pd.to_datetime(mean_rainfall_unstacked.date, format = "%d-%m-%y")

In [30]:
max_rainfall_unstacked.date = pd.to_datetime(max_rainfall_unstacked.date, format = "%d-%m-%y")

In [32]:
fatalities_unstacked.date = pd.to_datetime(fatalities_unstacked.date, format = "%Y-%m-%d")

In [33]:
full_features = cases_unstacked.merge(deaths_unstacked, on =['date', 'gov_iso']).merge(fatalities_unstacked, on =['date', 'gov_iso'], how = 'left').merge(mean_rainfall_unstacked, on =['date', 'gov_iso'], how = 'left').merge(max_rainfall_unstacked, on =['date', 'gov_iso'], how = 'left')

In [35]:
y = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/cholera_epi_data/y_normalized_groupby_week_df.csv')

In [40]:
full_features = full_features.sort_values('date')

In [41]:
y = y.sort_values('date')

In [42]:
full_features.head()

,gov_iso,date,new_cases,new_deaths,fatalities,mean_rainfall,max_rainfall
0,YE-AB,2017-05-23,0.901606,0.015179,0.0,0.000000,0.000000
2176,YE-HU,2017-05-23,0.564781,0.004239,0.0,14.555986,27.310665
3536,YE-MR,2017-05-23,0.000000,0.000000,0.0,0.000000,0.000000
3808,YE-MW,2017-05-23,3.025834,0.030040,0.0,17.384593,22.224941
4896,YE-SH,2017-05-23,0.000000,0.000000,0.0,0.000000,0.000000


In [43]:
y.head()

,gov_iso,date,week_1_cases,week_2_cases,week_2_to_4_cases,week_4_to_6_cases,week_6_to_8_cases
0,YE-AB,2017-05-23,6.976066,7.594340,18.482427,51.257999,65.009775
1325,YE-DH,2017-05-23,7.294939,9.135369,28.028750,44.058367,52.243076
4240,YE-SA,2017-05-23,32.503467,37.056994,95.130921,85.863835,65.281030
4770,YE-SH,2017-05-23,0.046047,0.015349,0.598610,2.118160,1.780482
795,YE-BA,2017-05-23,17.116538,11.683058,23.629375,29.137843,28.826720


In [73]:
y.date = pd.to_datetime(y.date, format = '%Y-%m-%d')

In [79]:
full_data = y.merge(full_features, on = ['gov_iso', 'date'], how = 'left')

In [97]:
extracted_features = extract_features(full_data.dropna()[full_features.columns], column_id = None, column_sort = 'date')

ValueError: You have to set the column_id which contains the ids of the different time series

In [93]:
impute(extracted_features)

 'fatalities__friedrich_coefficients__m_3__r_30__coeff_1'
 'fatalities__friedrich_coefficients__m_3__r_30__coeff_2'
 'fatalities__friedrich_coefficients__m_3__r_30__coeff_3'
 'fatalities__max_langevin_fixed_point__m_3__r_30'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_0'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_1'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_2'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_3'
 'max_rainfall__max_langevin_fixed_point__m_3__r_30'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_0'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_1'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_2'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_3'
 'mean_rainfall__max_langevin_fixed_point__m_3__r_30'
 'new_cases__friedrich_coefficients__m_3__r_30__coeff_0'
 'new_cases__friedrich_coefficients__m_3__r_30__coeff_1'
 'new_cases__friedrich_coefficients__m_3__r_30__coeff_2'
 'new_cases__

variable,fatalities__abs_energy,fatalities__absolute_sum_of_changes,"fatalities__agg_autocorrelation__f_agg_""mean""","fatalities__agg_autocorrelation__f_agg_""median""","fatalities__agg_autocorrelation__f_agg_""var""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,new_deaths__time_reversal_asymmetry_statistic__lag_1,new_deaths__time_reversal_asymmetry_statistic__lag_2,new_deaths__time_reversal_asymmetry_statistic__lag_3,new_deaths__value_count__value_-inf,new_deaths__value_count__value_0,new_deaths__value_count__value_1,new_deaths__value_count__value_inf,new_deaths__value_count__value_nan,new_deaths__variance,new_deaths__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
YE-AB,5.058424e-05,0.045156,0.001910,-0.016049,4.848242e-03,0.001093,-0.128861,-2.328604e-05,4.007039e-05,3.640141e-03,...,1.997230e-08,7.331172e-09,9.251240e-09,0.0,189.0,0.0,0.0,0.0,0.000026,0.0
YE-AD,2.093660e-05,0.018766,0.020918,0.003918,4.050364e-03,-0.000045,0.286350,3.756431e-05,2.810516e-05,3.043139e-04,...,7.715437e-09,1.557645e-08,2.358719e-08,0.0,187.0,0.0,0.0,0.0,0.000036,0.0
YE-AM,6.523904e-07,0.001901,0.000689,-0.006542,1.240979e-03,-0.000047,0.326795,8.586571e-06,5.552702e-06,-2.710505e-20,...,-1.615294e-07,-2.322723e-07,-5.232750e-07,0.0,136.0,0.0,0.0,0.0,0.000108,0.0
YE-BA,2.529550e-04,0.143764,0.000935,-0.013075,5.376275e-03,0.001581,0.295713,7.632561e-05,5.513325e-05,5.240804e-03,...,2.318667e-09,5.349049e-09,5.695607e-09,0.0,168.0,0.0,0.0,0.0,0.000020,0.0
YE-DA,1.258792e-04,0.076003,0.010157,-0.031189,8.196099e-03,0.000802,0.139619,3.327501e-05,5.276975e-05,2.368217e-03,...,7.072733e-07,-6.168427e-07,-5.021411e-07,0.0,162.0,0.0,0.0,0.0,0.000209,0.0
YE-DH,1.936197e-06,0.003056,-0.006777,-0.006726,5.679719e-07,-0.000015,0.175954,7.999146e-06,1.000690e-05,6.249921e-05,...,-1.870831e-08,-5.583754e-08,-3.717033e-08,0.0,114.0,0.0,0.0,0.0,0.000033,0.0
YE-HD-AL,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,214.0,0.0,0.0,0.0,0.000014,0.0
YE-HJ,8.399816e-06,0.028779,-0.002886,0.006945,3.823327e-03,0.000434,-0.039937,-1.426862e-06,7.982691e-06,5.222412e-04,...,-4.945288e-08,-1.089203e-06,-2.489136e-06,0.0,93.0,0.0,0.0,0.0,0.000357,0.0
YE-HU,3.134021e-07,0.002758,0.051825,0.019587,7.842467e-03,-0.000017,0.460007,6.542058e-06,2.823619e-06,3.228076e-05,...,1.086194e-07,9.869452e-08,8.707338e-08,0.0,66.0,0.0,0.0,0.0,0.000030,0.0


In [94]:
extracted_features

variable,fatalities__abs_energy,fatalities__absolute_sum_of_changes,"fatalities__agg_autocorrelation__f_agg_""mean""","fatalities__agg_autocorrelation__f_agg_""median""","fatalities__agg_autocorrelation__f_agg_""var""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","fatalities__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,new_deaths__time_reversal_asymmetry_statistic__lag_1,new_deaths__time_reversal_asymmetry_statistic__lag_2,new_deaths__time_reversal_asymmetry_statistic__lag_3,new_deaths__value_count__value_-inf,new_deaths__value_count__value_0,new_deaths__value_count__value_1,new_deaths__value_count__value_inf,new_deaths__value_count__value_nan,new_deaths__variance,new_deaths__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
YE-AB,5.058424e-05,0.045156,0.001910,-0.016049,4.848242e-03,0.001093,-0.128861,-2.328604e-05,4.007039e-05,3.640141e-03,...,1.997230e-08,7.331172e-09,9.251240e-09,0.0,189.0,0.0,0.0,0.0,0.000026,0.0
YE-AD,2.093660e-05,0.018766,0.020918,0.003918,4.050364e-03,-0.000045,0.286350,3.756431e-05,2.810516e-05,3.043139e-04,...,7.715437e-09,1.557645e-08,2.358719e-08,0.0,187.0,0.0,0.0,0.0,0.000036,0.0
YE-AM,6.523904e-07,0.001901,0.000689,-0.006542,1.240979e-03,-0.000047,0.326795,8.586571e-06,5.552702e-06,-2.710505e-20,...,-1.615294e-07,-2.322723e-07,-5.232750e-07,0.0,136.0,0.0,0.0,0.0,0.000108,0.0
YE-BA,2.529550e-04,0.143764,0.000935,-0.013075,5.376275e-03,0.001581,0.295713,7.632561e-05,5.513325e-05,5.240804e-03,...,2.318667e-09,5.349049e-09,5.695607e-09,0.0,168.0,0.0,0.0,0.0,0.000020,0.0
YE-DA,1.258792e-04,0.076003,0.010157,-0.031189,8.196099e-03,0.000802,0.139619,3.327501e-05,5.276975e-05,2.368217e-03,...,7.072733e-07,-6.168427e-07,-5.021411e-07,0.0,162.0,0.0,0.0,0.0,0.000209,0.0
YE-DH,1.936197e-06,0.003056,-0.006777,-0.006726,5.679719e-07,-0.000015,0.175954,7.999146e-06,1.000690e-05,6.249921e-05,...,-1.870831e-08,-5.583754e-08,-3.717033e-08,0.0,114.0,0.0,0.0,0.0,0.000033,0.0
YE-HD-AL,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,214.0,0.0,0.0,0.0,0.000014,0.0
YE-HJ,8.399816e-06,0.028779,-0.002886,0.006945,3.823327e-03,0.000434,-0.039937,-1.426862e-06,7.982691e-06,5.222412e-04,...,-4.945288e-08,-1.089203e-06,-2.489136e-06,0.0,93.0,0.0,0.0,0.0,0.000357,0.0
YE-HU,3.134021e-07,0.002758,0.051825,0.019587,7.842467e-03,-0.000017,0.460007,6.542058e-06,2.823619e-06,3.228076e-05,...,1.086194e-07,9.869452e-08,8.707338e-08,0.0,66.0,0.0,0.0,0.0,0.000030,0.0


In [95]:
features_filtered = select_features(extracted_features, full_data.dropna().week_4_to_6_cases)

ValueError: Index of X must be a subset of y's index

In [102]:
full_data.dropna()[full_features.columns]

,gov_iso,date,new_cases,new_deaths,fatalities,mean_rainfall,max_rainfall
0,YE-AB,2017-05-23,0.901606,0.015179,0.000000,0.000000,0.000000
1,YE-DH,2017-05-23,1.048868,0.009428,0.000000,0.454375,2.987849
2,YE-SA,2017-05-23,5.665804,0.005107,0.000000,0.000000,0.000000
3,YE-SH,2017-05-23,0.000000,0.000000,0.000000,0.000000,0.000000
4,YE-BA,2017-05-23,2.484196,0.004787,0.000000,0.124548,1.058524
5,YE-JA,2017-05-23,0.247306,0.000000,0.000000,0.000000,0.000000
6,YE-LA,2017-05-23,0.345464,0.002159,0.000000,0.068845,1.170363
7,YE-AM,2017-05-23,3.967424,0.021359,0.000000,1.432684,5.729936
8,YE-SN,2017-05-23,1.375641,0.006141,0.000192,3.610576,20.015570
9,YE-RA,2017-05-23,0.843773,0.015920,0.000000,12.988092,23.290703


In [103]:
full_data.dropna().week_4_to_6_cases

0       51.257999
1       44.058367
2       85.863835
3        2.118160
4       29.137843
5       17.612988
6       17.445921
7       92.430835
8       31.592361
9       37.551865
10       0.000000
11      58.737408
12      62.769676
13      12.760157
14      19.025484
15      14.324474
16      28.290273
17       2.672977
18      30.935804
19       4.500000
20      85.859416
21      14.658214
22      44.718329
23      83.369724
24      20.546807
25       2.118160
26      26.481333
27      28.032838
28      13.948991
29      18.457446
          ...    
4506     0.019814
4507    11.912393
4508     9.940466
4509     0.000000
4510     0.000000
4511     8.272255
4512    12.412101
4513     0.013018
4514     3.492994
4515     8.508011
4516     9.708340
4517     0.000000
4518     0.000000
4519     0.010125
4520     1.327006
4521     2.065931
4522     4.423942
4523     0.893633
4524     0.158965
4525     7.913653
4526     0.000000
4527     0.000000
4528    11.185197
4529     0.413735
4530     0

In [104]:
features_filtered_direct = extract_relevant_features(full_data.dropna()[full_features.columns], full_data.dropna().week_4_to_6_cases.values,
                                                     column_id='gov_iso', column_sort='date')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 18/18 [00:16<00:00,  1.09it/s]
 'fatalities__friedrich_coefficients__m_3__r_30__coeff_1'
 'fatalities__friedrich_coefficients__m_3__r_30__coeff_2'
 'fatalities__friedrich_coefficients__m_3__r_30__coeff_3'
 'fatalities__max_langevin_fixed_point__m_3__r_30'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_0'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_1'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_2'
 'max_rainfall__friedrich_coefficients__m_3__r_30__coeff_3'
 'max_rainfall__max_langevin_fixed_point__m_3__r_30'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_0'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_1'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_2'
 'mean_rainfall__friedrich_coefficients__m_3__r_30__coeff_3'
 'mean_rainfall__max_langevin_fixed_point__m_3__r_30'
 'new_cases__friedrich_coefficients__m_3__r_30__coeff_0'
 'new_c

ValueError: Length of passed values is 4536, index implies 21

In [101]:
features_filtered_direct

NameError: name 'features_filtered_direct' is not defined